### Imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import platform
from pathlib import Path
from os import environ
if platform.system() == "Darwin": # Nat laptop
    sys.path.extend(['/Users/nkinsky/Documents/UM/GitHub/NeuroPy'])
    sys.path.extend(['/Users/nkinsky/Documents/UM/GitHub/sleep_deprivation/DataPaths/'])
    plot_dir = Path("/Users/nkinsky/Documents/UM/Working/Bapun_Data/plots")
else:
    if environ["HOSTNAME"] == "lnx00004": # Nat Linux computer
        sys.path.extend(['/data/GitHub/NeuroPy'])
        sys.path.extend(['/data/GitHub/sleep_loss_hippocampal_replay/analysis_and_figure_code/DataPaths'])
        plot_dir = Path('/data/Clustering/plots')
    else:  #BG computer
        sys.path.extend(['/home/nkinsky/Documents/GitHub/NeuroPy'])
        sys.path.extend(['/home/nkinsky/Documents/GitHubPrivate/pythonprogs/DataPaths/'])

group_data_dir = Path('/data/Clustering/grpdata')
    
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import pingouin as pg
import scipy.signal as sg
import matplotlib.pyplot as plt
from neuropy.utils.mathutil import min_max_scaler
from scipy.ndimage import gaussian_filter1d, gaussian_filter

from neuropy import plotting
from neuropy.analyses.placefields import Pf1D
from neuropy.core.position import Position
from neuropy.plotting.ratemaps import plot_ratemap
from neuropy.plotting.figure import Fig
from neuropy.utils.misc import flatten

import subjects
from stats_utils import bootstrap_resample

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

/home/nkinsky/miniforge3/envs/NeuroPy/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


## Define useful functions

In [ ]:
def pf_corr_maze_remaze(row, nbins_each_sesh=50):
    """Calculate PF correlation for each row/neuron. 
    
    row: a DataFrame row of values (or 1d ndarray) where the first nbins_each_sesh bins 
    are firing rates for MAZE in each bin and the next nbins_each_sesh are for reMAZE"""
    maze_tc = row.values[0:nbins_each_sesh]
    remaze_tc = row.values[nbins_each_sesh:]
    

    return np.corrcoef([maze_tc, remaze_tc])[0, 1]

In [ ]:
def get_pf_loc(row, nbins_each_sesh=50, end_prop=0.125):
    """Calculate placefield peak location
    
    row: a DataFrame row of values (or 1d ndarray) where the first nbins_each_sesh bins 
    are firing rates for MAZE in each bin and the next nbins_each_sesh are for reMAZE
    
    end_prop: propotion of the track corresponding to end - fields with locations less than end_prop or 
    more than 1 - end_prop are classified as 'end' pfs while the rest are classified as 'middle' pfs"""

    # Get location on track
    maze_tc = row.values[0:nbins_each_sesh]
    imax = np.argmax(maze_tc)
    
    if (imax < (nbins_each_sesh * end_prop)) | (imax > (nbins_each_sesh - nbins_each_sesh * end_prop)):
        loc = "end"
    else:
        loc = "middle"

    return loc

In [ ]:
def pf_corr_maze_remaze_ends(row, nbins_each_sesh=50, end_prop=0.125):
    """Calculate PF correlation for each row/neuron but only consider neurons with peaks in at the end of the tracks,
    within `end_prop` of the end of the track"""

    # Calculate correlations
    pf_corr_row = pf_corr_maze_remaze(row, nbins_each_sesh=nbins_each_sesh)

    # Get location on track
    maze_tc = row.values[0:nbins_each_sesh]
    imax = np.argmax(maze_tc)

    if (imax < (nbins_each_sesh * end_prop)) | (imax > (nbins_each_sesh - nbins_each_sesh * end_prop)):
        return pf_corr_row
    else:
        return np.nan


In [ ]:
def pf_corr_maze_remaze_middle(row, nbins_each_sesh=50, end_prop=0.125):
    """Calculate PF correlation for each row/neuron but only consider neurons with peaks in at the end of the tracks,
    within `end_prop` of the end of the track"""

    # Calculate correlations
    pf_corr_row = pf_corr_maze_remaze(row, nbins_each_sesh=nbins_each_sesh)

    # Get location on track
    maze_tc = row.values[0:nbins_each_sesh]
    imax = np.argmax(maze_tc)

    if (imax > (nbins_each_sesh * end_prop)) & (imax < (nbins_each_sesh - nbins_each_sesh * end_prop)):
        return pf_corr_row
    else:
        return np.nan


In [ ]:
def get_imax(row):
    maze_tc = row.values[0:nbins_each_sesh]
    imax = np.argmax(maze_tc)

    return imax

In [ ]:
def delta_rate_index(row, nbins_each_sesh=50, use_same_peak_bin=False):
    """Calculate change in max firing rate from MAZE to REMAZE for each row/neuron"""
    maze_tc = row.values[0:nbins_each_sesh]
    remaze_tc = row.values[nbins_each_sesh:]
    
    maze_max = np.nanmax(maze_tc)
    remaze_max = np.nanmax(remaze_tc)
    if use_same_peak_bin:
        imax = np.argmax(maze_tc)
        remaze_max = remaze_tc[imax]
    
    return (maze_max - remaze_max) / (maze_max + remaze_max)

In [ ]:
def delta_pf_peak(row, nbins_each_sesh=50):
    maze_tc = row.values[0:nbins_each_sesh]
    remaze_tc = row.values[nbins_each_sesh:]

    return np.argmax(maze_tc) - np.argmax(remaze_tc)

In [ ]:
def get_number_of_fields(pf_peak_or_width_list):
    nfields = [cell_peaks.size if ~np.all(np.isnan(cell_peaks)) else 0 for cell_peaks in pf_peak_or_width_list]

    return nfields

In [ ]:
df_bs

In [ ]:
pd.read_csv(group_data_dir / "remaze_fr_bootstrap.csv", index_col=0)

In [ ]:
pd.melt(df_bs, id_vars=["grp"], value_vars=["MAZE", "reMAZE"], var_name="session", value_name="mean_frate")

In [ ]:
_, ax = plt.subplots()
df2 = pd.melt(df_bs, id_vars=["grp"], value_vars=["MAZE", "reMAZE"], var_name="session", value_name="mean_frate")
sns.boxplot(data=df2, x="grp", y="mean_frate", hue="session", ax=ax)

In [ ]:
fr_df_rs.neuron_type.unique()

In [ ]:
fr_df_rs = fr_df.pivot_table(index=["grp", "session", "neuron_id", "neuron_type"], 
                             columns="session_type", 
                             values="mean_frate").reset_index().drop(columns="neuron_id")

func = lambda df: df.drop(["session"], axis=1).groupby(["grp"]).mean().reset_index()
df_use = fr_df_rs[fr_df_rs.neuron_type == "inter"].drop(columns="neuron_type")
df_bs = bootstrap_resample(df_use, level="both", n_iter=1000, apply=func)
df_bs

In [ ]:
pd.melt(df_bs, id_vars=["grp"], value_vars=["MAZE", "reMAZE"], var_name="session", value_name="mean_frate")

In [ ]:
_, ax = plt.subplots()
df2 = pd.melt(df_bs, id_vars=["grp"], value_vars=["MAZE", "reMAZE"], var_name="session", value_name="mean_frate")
sns.boxplot(data=df2, x="grp", y="mean_frate", hue="session", ax=ax)

In [ ]:
# Use to look at t-tests for each session - not that meaningful due to generally low n for most sessions with interneurons
fr_use = fr_df[fr_df.neuron_type == "inter"]
for sid in fr_use.session.unique():
    print(pg.ttest(fr_use[(fr_use.session == sid) & (fr_use.session_type == "MAZE")]["mean_frate"],
         fr_use[(fr_use.session == sid) & (fr_use.session_type == "reMAZE")]["mean_frate"],
         paired=True))

In [ ]:
fr_use = fr_df[fr_df.neuron_type == "inter"]

# Significant drop of inhibition for SD group but not NSD group with paired t-test (for stable neurons only, doesn't hold if we include all)

print(pg.ttest(fr_use[(fr_use.grp == "SD") & (fr_use.session_type == "MAZE")]["mean_frate"],
         fr_use[(fr_use.grp == "SD") & (fr_use.session_type == "reMAZE")]["mean_frate"],
         paired=True))

# sstats.ks_2samp(fr_use[(fr_use.grp == "SD") & (fr_use.session_type == "MAZE")]["mean_frate"],
#                 fr_use[(fr_use.grp == "SD") & (fr_use.session_type == "reMAZE")]["mean_frate"])

# Distribution of interneuron firing rates is different between groups as well (stable neurons only, doesn't hold if we include all)
stats.ks_2samp(fr_use[(fr_use.grp == "SD") & (fr_use.session_type == "MAZE")]["delta_frate_maze_remaze"],
                fr_use[(fr_use.grp == "NSD") & (fr_use.session_type == "MAZE")]["delta_frate_maze_remaze"])

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(4, 6))
for ax, ntype, nname in zip(axs, ["pyr", "inter"], ["Pyr. Cells", "Interneurons"]):
    fr_use = fr_df[fr_df.neuron_type == ntype]
    print(f"{ntype} below")
    print(pg.ttest(fr_use[(fr_use.grp == "SD") & (fr_use.session_type == "MAZE")]["mean_frate"],
         fr_use[(fr_use.grp == "SD") & (fr_use.session_type == "reMAZE")]["mean_frate"],
         paired=True))

    for idg, (gname, a) in enumerate(zip(["NSD", "SD"], ax)):
        pal_use = sns.color_palette()[idg:]
        sns.lineplot(data=fr_df[(fr_df.neuron_type == ntype) & (fr_df.grp == gname)], x="session_type", y="mean_frate",
                     hue="grp", units="nid_unique", estimator=None, legend=False, palette=pal_use, ax=a)
        sns.stripplot(data=fr_df[(fr_df.neuron_type == ntype) & (fr_df.grp == gname)], x="session_type", y="mean_frate", 
                      hue="grp", dodge=True, jitter=True, legend=False, palette=pal_use, ax=a)
        a.set_title(f"{gname}: {nname}")
        a.set_ylabel("Mean FR (Hz)")
        a.set_xlabel("")

sns.despine(fig=fig)

fig.savefig(plot_dir / "fr_stripplot_maze_v_remaze.pdf")

In [ ]:
# Create combined colormap, use cmap3[0::2] to plot dark colors and cmap[1::2] to plot same colors but in light / pastel
cmap1 = sns.color_palette(as_cmap=True)
cmap2 = sns.color_palette("pastel", as_cmap=True)
cmap3 = []
for c1, c2 in zip(cmap1, cmap2):
    cmap3.extend([c1, c2])


In [ ]:
for ntype, nname in zip(["pyr", "inter"], ["Pyramidal Cells", "Interneurons"]):
    fig, axs = plt.subplots(2, 2, figsize=(9, 9))
    fig.suptitle(nname)

    ax = axs[0]
    sns.stripplot(data=fr_df[fr_df.neuron_type == ntype], x="grp", y="mean_frate", hue="session_type", dodge=True, ax=ax[0])
    # sns.lineplot(data=fr_df[fr_df.neuron_type == ntype], x="grp", y="mean_frate", hue="session_type", units="unique_nid", estimator=None, ax=ax[0])
    sns.violinplot(data=fr_df[fr_df.neuron_type == ntype], x="grp", y="mean_frate", hue="session_type", split=True, ax=ax[1])
    [a.set_ylabel("Mean FR (Hz)") for a in ax];
    
    ax2 = axs[1]
    # sns.violinplot(data=fr_df[fr_df.neuron_type == ntype], x="neuron_type", y="delta_frate_maze_remaze", hue="grp", split=True, ax=ax2[0])
    sns.stripplot(data=fr_df[fr_df.neuron_type == ntype], y="delta_frate_maze_remaze", hue="grp", dodge=True, ax=ax2[0])
    ax2[0].set_ylabel(r"$FR_{MAZE} - FR_{reMAZE}$")
    ax2[0].set_xlabel("")
    sns.ecdfplot(data=fr_df[fr_df.neuron_type == ntype], x="delta_frate_maze_remaze", hue="grp", ax=ax2[1])
    ax2[1].set_xlabel(r"$FR_{MAZE} - FR_{reMAZE}$")

    sns.despine(fig=fig)
    fig.savefig(plot_dir / f"FR_{ntype}delta_comprenhensive.pdf")